### The solution to the competition https://zindi.africa/competitions/classification-for-landslide-detection demonstrates the process of creating and training a deep learning model for binary classification of landslides based on multispectral data and Sentinel-2.
### Competition metric - f1

### Installing the comet_ml library. Comet ML is a platform for tracking, comparing, and optimizing machine learning experiments

In [ ]:
# Import necessary libraries
!pip install comet_ml > /dev/null 2>&1

In [ ]:
import comet_ml
COMET_API_KEY = "My CometML"
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Input, GlobalAveragePooling2D
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
from scipy.ndimage import rotate, shift, zoom
data = "/kaggle/input/slideandseekclasificationlandslidedetectiondataset"

### Defining data paths and preloading the train and test CSV files

In [ ]:
train_csv_path = f'{data}/Train.csv'
test_csv_path = f'{data}/Test.csv'
train_data_path = f'{data}/train_data/train_data'
test_data_path = f'{data}/test_data/test_data'

train_df = pd.read_csv(train_csv_path)
print("Train.csv:")
print(train_df.head())


### Visualization of class distribution (No landslide effects/after landslide)

In [ ]:
label_counts = train_df['label'].value_counts()
labels = ['No Landslide', 'Landslide']

plt.figure(figsize=(6, 4))
plt.bar(labels, label_counts.values, color=['skyblue', 'salmon'])
plt.xlabel("Class Label")
plt.ylabel("Frequency")
plt.title("Distribution of Labels in Training Set")
plt.show()

### As can be seen from the graph, there is a strong imbalance of classes No Landslide ~ 6000 images, Landslide ~ 1400 images
### This problem will be solved further using Focal loss

### Image loading and normalization function, as well as their visualization
### The images consist of 12 bands (4 optical from Sentinel-2 and 8 SAR from Sentinel-1) and are 64x64 in size. For each band, a robust percentile normalization (2nd and 98th) is performed to make the pixel values in the range [0, 1] and reduce the influence of outliers that may be present in the satellite data. Normalization to 1/99th percentile and minmax scaling (commented code) were also tested, but they gave worse results.

In [ ]:
def load_and_normalize_npy_image(image_id, folder_path):
    image_path = os.path.join(folder_path, f"{image_id}.npy")
    
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")
    
    img = np.load(image_path).astype(np.float32)

    normalized_img = img
    for band in range(img.shape[2]):
        band_data = img[:, :, band]
        
        p2, p98 = np.percentile(band_data, (2, 98))
        normalized_img[:, :, band] = np.clip((band_data - p2) / (p98 - p2 + 1e-6), 0, 1)
        
    #     p1, p99 = np.percentile(band_data, (1, 99))

    #     clipped_data = np.clip(band_data, p1, p99)
    #     mean_val = np.mean(clipped_data)
    #     std_val = np.std(clipped_data)
        
    #     if std_val > 0:
    #         normalized_img[:, :, band] = (clipped_data - mean_val) / std_val
    #     else:
    #         normalized_img[:, :, band] = clipped_data - mean_val
    
    # for band in range(img.shape[2]):
    #     band_data = normalized_img[:, :, band]
    #     min_val, max_val = np.min(band_data), np.max(band_data)
    #     if max_val > min_val:
    #         normalized_img[:, :, band] = (band_data - min_val) / (max_val - min_val)
    
    return normalized_img

band_descriptions = [
    "Red", "Green", "Blue", "Near Infrared",
    "Descending VV (Vertical-Vertical)", "Descending VH (Vertical-Horizontal)",
    "Descending Diff VV", "Descending Diff VH",
    "Ascending VV (Vertical-Vertical)", "Ascending VH (Vertical-Horizontal)",
    "Ascending Diff VV", "Ascending Diff VH"
]

example_ids = train_df['ID'].sample(2,random_state = 42).values

for image_id in example_ids:
    img_normalized = load_and_normalize_npy_image(image_id, train_data_path)

    fig, axes = plt.subplots(3, 4, figsize=(20, 12))
    fig.suptitle(f"Sample Image ID: {image_id} - All 12 Bands", fontsize=16)

    for band in range(12):
        row = band // 4
        col = band % 4
        axes[row, col].imshow(img_normalized[:, :, band], cmap='gray')
        axes[row, col].set_title(f"Band {band + 1}: {band_descriptions[band]}")
        axes[row, col].axis('off')

    plt.subplots_adjust(wspace=0.3, hspace=0.4)
    plt.show()

### This is what two 12-channel images look like (the first three channels are the usual RGB, then infrared and other types of radiation produced and received by Sentinel 2.

### Defining a data generator for loading data in batches with random examples from train and applying random augmentations (15 degree rotation, width and height shifts, scaling, reflections and filling in gaps at the edges.)

In [ ]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

class LandslideDataGenerator(Sequence):
    def __init__(self, df, data_path, batch_size=32, augment=False, shuffle=True):
        self.df = df.reset_index(drop=True)
        self.data_path = data_path
        self.batch_size = batch_size
        self.augment = augment
        self.shuffle = shuffle
        self.on_epoch_end()

        if self.augment:
            self.augmentor = ImageDataGenerator(
                rotation_range=15,
                width_shift_range=0.1,
                height_shift_range=0.1,
                shear_range=0.1,
                zoom_range=0.1,
                horizontal_flip=True,
                vertical_flip=True,
                fill_mode='reflect' 
            )
        else:
            self.augmentor = None

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indexes)
        
    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_df = self.df.iloc[batch_indexes]
        
        batch_X = []
        batch_y = []
        
        for _, row in batch_df.iterrows():
            image_id = row['ID']
            label = row['label']
           
            image = load_and_normalize_npy_image(image_id, self.data_path)
            batch_X.append(image)
            batch_y.append(label)
        
        batch_X = np.array(batch_X, dtype=np.float32)
        batch_y = np.array(batch_y, dtype=np.float32).reshape(-1, 1)

        if self.augment and self.augmentor:
            batch_X = next(self.augmentor.flow(batch_X, batch_size=len(batch_X), shuffle=False))
        return batch_X, batch_y


In [ ]:
folder_path = '/kaggle/input/slideandseekclasificationlandslidedetectiondataset/train_data/train_data/'

batch_size = 16

train_df_split, val_df_split = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['label'])

train_gen = LandslideDataGenerator(train_df_split, folder_path, batch_size=batch_size, augment=True, shuffle=True)
val_gen = LandslideDataGenerator(val_df_split, folder_path, batch_size=batch_size, augment=False, shuffle=False)

X_batch, y_batch = train_gen[0]

X_batch.shape, y_batch.shape

### As we can see in the X batch there are 16 images 64x64 with 12 channels and in the label(y) batch there are 16 classes corresponding to the images

### Output y_batch

In [ ]:
y_batch

### Definition of Focal Loss
### Focal Loss was designed to address class imbalance in problems where one class is significantly dominant over another (like in landslides). It modifies the standard cross-entropy loss by decreasing the weight of "easy" (well-classified) examples and increasing the weight of "hard" (badly-classified) examples, especially for the minority class.

### gamma (focus parameter): controls how much "easy" examples are suppressed. Higher gamma -> stronger focus on hard ones.

### alpha (balancing factor): For a rare positive class, alpha is usually set high (e.g. 0.75 or 0.9 depending on the degree of imbalance). In the current implementation, alpha=None by default, which means automatic weight calculation (based on the number of class examples in the train).

In [ ]:
def focal_loss(gamma=2.0, alpha=0.75):
    """
    Focal Loss for binary classification.

    Parameters:
        gamma (float): Focusing parameter; typically set to 2.0.
        alpha (float): Balancing factor; typically set to 0.25.

    Returns:
        Binary Focal Loss function.
    """
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())

        if alpha is None:

            pos_weight = K.mean(1 - y_true)
            neg_weight = K.mean(y_true)
            alpha_t = tf.where(K.equal(y_true, 1), pos_weight, neg_weight)
        else:
            alpha_t = tf.where(K.equal(y_true, 1), alpha, 1 - alpha)

        p_t = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)

        fl = -alpha_t * K.pow(1 - p_t, gamma) * K.log(p_t)
        return K.mean(fl)
    
    return focal_loss_fixed

### Standard metrics Precision, Recall and F1-score

In [ ]:
def precision_m(y_true, y_pred):
    y_pred_bin = tf.cast(y_pred >= 0.5, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    true_positives = tf.reduce_sum(y_true * y_pred_bin)
    predicted_positives = tf.reduce_sum(y_pred_bin)
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall_m(y_true, y_pred):
    y_pred_bin = tf.cast(y_pred >= 0.5, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    true_positives = tf.reduce_sum(y_true * y_pred_bin)
    possible_positives = tf.reduce_sum(y_true)
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

### Then more than 50 experiments were conducted on various architectures: simple CNN models of different depths, with different activation functions (RelU LeakyReLU, ELU) with different parameters for dropout and convolution sizes, with different hyperparameters lr, batch_size, etc. ### F1 plot on the validation set of all Cnn experiments:
![all_cnn](https://github.com/MakarRybkin/Slide_and_seek_classification/raw/master/images_for_report/all_cnn_f1.png)
### With this approach, we were able to achieve f1 = 0.8054
### F1 plot on the validation set of the best Cnn experiment:
![best_cnn.png](https://github.com/MakarRybkin/Slide_and_seek_classification/raw/master/images_for_report/best_cnn.png)
### An example of one of the CNN models can be seen in the commented code below

In [ ]:
# model = Sequential([
#
#     Input(shape=X_batch.shape[1:]),
#     Conv2D(32, (3, 3), activation='relu', padding='same'),
#     BatchNormalization(),
#     Conv2D(32, (3, 3), activation='relu', padding='same'),
#     MaxPooling2D((2, 2)),
#     Dropout(0.1),
    
#
#     Conv2D(64, (3, 3), activation='relu', padding='same'),
#     BatchNormalization(),
#     Conv2D(64, (3, 3), activation='relu', padding='same'),
#     MaxPooling2D((2, 2)),
#     Dropout(0.1),
    
#
#     Conv2D(128, (3, 3), activation='relu', padding='same'),
#     BatchNormalization(),
#     Conv2D(128, (3, 3), activation='relu', padding='same'),
#     MaxPooling2D((2, 2)),
#     Dropout(0.15),
    
#
#     Conv2D(256, (3, 3), activation='relu', padding='same'),
#     BatchNormalization(),
#     Conv2D(256, (3, 3), activation='relu', padding='same'),
#     GlobalAveragePooling2D(),  # Better than Flatten for reducing parameters
#     Dropout(0.15),
    
#
#     Dense(64, activation='relu'),
#     BatchNormalization(),
#     Dropout(0.3),
#     Dense(1, activation='sigmoid')
# ])

# optimizer = AdamW(
#     learning_rate=0.005,  
#     weight_decay=0.01,
#     beta_1=0.9,
#     beta_2=0.999
# )

# model.compile(
#         optimizer=optimizer,
#         loss=focal_loss(gamma=2.0, alpha=None),  # Auto-balanced
#         metrics=[f1_m, precision_m, recall_m]
#     )


### After that, experiments were also conducted with different parameters and hyperparameters for more complex ready-made models (EfficientNet(B0 - B7) EfficientNetV2L, different depths of ResNet and DenseNet followed by the head of the model, consisting of GlobalAveragePooling2D() to reduce the dimensionality after the output and several layers of FCNN

### The f1 graph on the validation set of all experiments with these architectures:

![all_ef_res_dense.png](https://github.com/MakarRybkin/Slide_and_seek_classification/raw/master/images_for_report/all_ef_res_dense.png)
### As a result, the best models were EfficientNet B4 (best f1 ~ 0.8389), ResNet50 (best f1 ~ 0.838) DenseNet_169 (best f1 ~ 0.841)

### f1 plot on the validation set for the best experiment with EfficientNetB4 :
![best_eff_b4.png](https://github.com/MakarRybkin/Slide_and_seek_classification/raw/master/images_for_report/best_eff_b4.png)

### f1 plot on the validation set for the best experiment with ResNet50 :
![best_resnet.png](https://github.com/MakarRybkin/Slide_and_seek_classification/raw/master/images_for_report/best_resnet.png)

### f1 plot on the validation set for the best experiment with DenseNet169:
![best_densenet.png](https://github.com/MakarRybkin/Slide_and_seek_classification/raw/master/images_for_report/best_densenet.png)
### The commented code for training one of the models from this series can be seen below

In [ ]:
# from tensorflow.keras.applications import EfficientNetB3 , EfficientNetV2L , DenseNet169, ResNet50
# from tensorflow.keras.layers import LeakyReLU, ELU , ReLU
# input_shape = X_batch.shape[1:] 

# base_efficientnet = EfficientNetB3(weights=None, include_top=False, input_shape=input_shape)

# # densnet_169 = DenseNet169(weights=None, include_top=False, input_shape=input_shape)

# # resnet50 = ResNet50(weights=None, include_top=False, input_shape=input_shape)

# model = Sequential([
#     base_efficientnet,
#     GlobalAveragePooling2D(),
    
#     Dense(512), 
#     BatchNormalization(),
#     LeakyReLU(alpha = 0.01), 
#     Dropout(0.3),
    
#     Dense(256),
#     BatchNormalization(),
#     LeakyReLU(alpha = 0.01),
#     Dropout(0.2),
    
#     Dense(64),
#     BatchNormalization(),
#     LeakyReLU(alpha = 0.01),
#     Dropout(0.2),
#     Dense(1, activation='sigmoid')
# ])


# optimizer_efficientnet = tf.keras.optimizers.AdamW(
#     learning_rate=0.001,
#     weight_decay=0.01,
#     beta_1=0.9,
#     beta_2=0.999
# )

# model.compile(
#     optimizer=optimizer_efficientnet,
#     loss=focal_loss(gamma=2.0, alpha=None), 
#     metrics=[f1_m, precision_m, recall_m] 
# )



### Definition of constants and hyperparameters

### As a result of previous experiments, the best hyperparameters and architectures (from those studied) were identified; they will be used below for combining into an ensemble

In [ ]:
from tensorflow.keras.applications import EfficientNetB5 , EfficientNetV2L , DenseNet169, ResNet50
from tensorflow.keras.layers import LeakyReLU, ELU , ReLU
MODEL_TYPES_FOR_ENSEMBLE = [
    'EfficientNetB5',
    'DenseNet169',
    'ResNet50'
]
NUM_RUNS_PER_MODEL_TYPE = 2
BASE_RANDOM_SEED = 100


LEARNING_RATE = 0.001
BATCH_SIZE_PHASE1 = 16
EPOCHS_PHASE1 = 30
BATCH_SIZE_PHASE2 = 32
EPOCHS_PHASE2_MAX = 70
PATIENCE_EARLY_STOPPING = 25
PATIENCE_REDUCE_LR = 6
MIN_LR = 1e-8
DROPOUT_RATE_DENSE1 = 0.2
DROPOUT_RATE_DENSE2 = 0.15
DROPOUT_RATE_DENSE3 = 0.15

COMET_PROJECT_NAME = "landslide-ensemble-custom-loop"

### Function to create and compile a single model (will be used below for each model in the ensemble). Depending on the model_type (EfficientNetB5, DenseNet169 or ResNet50), the corresponding architecture is loaded from tensorflow.keras.applications. weights=None specifies that the models are loaded without pre-trained ImageNet weights. optimizer and compiler for the model.

### The following layers are added to the base model for classification:
#### GlobalAveragePooling2D(): Reduces the dimensionality of spatial features to a single vector by averaging the values over the width and height.

#### Dense (fully connected layers): Two fully connected layers (256 and 64 neurons) with LeakyReLU activation functions for nonlinearity.

#### BatchNormalization(): Normalizes activations to speed up training and improve stability.

#### Dropout(): Uses neuron pruning to prevent overfitting.

#### Final Dense(1, activation='sigmoid'): Single neuron output layer with sigmoid activation for binary classification (output in range [0, 1], interpreted as probability of landslide).

#### Uses tf.keras.optimizers.AdamW. This is a variant of Adam optimizer that explicitly takes weight decay into account, which helps prevent overfitting.

### Compiling the model: The model is compiled with:

#### optimizer: AdamW defined above.

#### loss: Custom focal_loss function to deal with class imbalance.

#### metrics: Custom metrics f1_m, precision_m, recall_m to evaluate performance.

In [ ]:
def create_compiled_model(model_type, input_shape):
    base_model = None
    if model_type == 'EfficientNetB5':
        base_model = EfficientNetB5(weights=None, include_top=False, input_shape=input_shape)
    elif model_type == 'DenseNet169':
        base_model = DenseNet169(weights=None, include_top=False, input_shape=input_shape)
    elif model_type == 'ResNet50':
        base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),

        Dense(256),
        BatchNormalization(),
        LeakyReLU(alpha=0.01), 
        Dropout(DROPOUT_RATE_DENSE2),
        
        Dense(64),
        BatchNormalization(),
        LeakyReLU(alpha=0.01), 
        Dropout(DROPOUT_RATE_DENSE3),
        Dense(1, activation='sigmoid')
    ])

    optimizer = tf.keras.optimizers.AdamW(
        learning_rate=LEARNING_RATE,
        weight_decay=0.01,
        beta_1=0.9,
        beta_2=0.999
    )

    model.compile(
        optimizer=optimizer,
        loss=focal_loss(gamma=2.0, alpha=None),
        metrics=[f1_m, precision_m, recall_m]
    )
    return model

### Main model ensemble training loop

### Initialization: Defines variables to store paths to trained models and metadata.

#### Double loop: The code loops through each model_type (EfficientNetB5, DenseNet169, ResNet50) and for each type performs NUM_RUNS_PER_MODEL_TYPE (in this case 2) independent training runs. This creates diversity in the ensemble.

#### Reproducibility: Each run sets a new current_seed for tf.random, np.random, and os.environ['PYTHONHASHSEED'] to ensure that random initialization and operations are unique, but it is still possible to replay each individual run given its seed.

#### Comet ML integration:

#### A new comet_ml.Experiment object is created before each new experiment.

#### Sets the name of the experiment that will be displayed in the Comet ML dashboard.

#### This allows you to track metrics, loss, and other data for each model run separately.

#### Two-Phase Training: Each model is trained in two phases:

#### Phase 1: Train with BATCH_SIZE_PHASE1=16. Smaller batches can help converge faster at the beginning of training.

#### Phase 2: Continue training with BATCH_SIZE_PHASE2=32. Larger batches can help provide a more stable gradient and better generalization in the later stages.

#### Callbacks: The same callbacks are used for both training phases:

#### ModelCheckpoint: Saves the best version of the model based on the val_f1_m metric (F1 on the validation set). save_best_only=True ensures that only the best model is saved.

#### EarlyStopping: Stops training if val_f1_m does not improve within PATIENCE_EARLY_STOPPING=25 epochs. restore_best_weights=True loads the best epoch weights after stopping.

#### ReduceLROnPlateau: Reduces the learning rate if val_f1_m does not improve within PATIENCE_REDUCE_LR=6 epochs. This helps the model escape from local minima.

#### Model training: model.fit() runs the training process using the generated data generators and callbacks.

#### Saving models: After both phases of training (or early stopping) are complete, the best version of each trained model is saved to disk as h5.keras or keras in the ensemble_models_custom folder. The paths and metadata of the models are recorded for later use in the ensemble.

#### Ending a Comet ML experiment: experiment.end() closes the current Comet ML experiment, saving all logs.

In [ ]:
input_shape = X_batch.shape[1:] 

trained_model_paths = [] 
model_metadata = []      

total_runs = len(MODEL_TYPES_FOR_ENSEMBLE) * NUM_RUNS_PER_MODEL_TYPE
run_counter = 0

for model_type in MODEL_TYPES_FOR_ENSEMBLE:
    for i in range(NUM_RUNS_PER_MODEL_TYPE):
        run_counter += 1
        print(f"\n--- Обучение модели {model_type} (Запуск {i+1}/{NUM_RUNS_PER_MODEL_TYPE}) ---")

        current_seed = BASE_RANDOM_SEED + run_counter * 10
        tf.random.set_seed(current_seed)
        np.random.seed(current_seed)
        os.environ['PYTHONHASHSEED'] = str(current_seed)

        if 'experiment' in globals() and isinstance(globals()['experiment'], comet_ml.Experiment):
            if not globals()['experiment'].ended:
                globals()['experiment'].end()
        experiment = comet_ml.Experiment(
            api_key=COMET_API_KEY,
            project_name="Classification for Landslide Detection Ensamble", 
            auto_output_logging="simple"
        )
        experiment.flush()
        experiment_name = f"{model_type}_run_{i+1}_seed_{current_seed}"
        experiment.set_name(experiment_name)

        print(f"--- Фаза 1: Обучение с Batch Size {BATCH_SIZE_PHASE1} на {EPOCHS_PHASE1} эпох ---")
        train_gen_phase1 = LandslideDataGenerator(
            train_df_split, folder_path, batch_size=BATCH_SIZE_PHASE1, augment=True, shuffle=True)
        val_gen_phase1 = LandslideDataGenerator(
            val_df_split, folder_path, batch_size=BATCH_SIZE_PHASE1, augment=False, shuffle=False)

        model = create_compiled_model(model_type, input_shape)

        callbacks_phase1 = [
            ModelCheckpoint(
                f"best_model{experiment_name}.h5.keras",
                monitor='val_f1_m',
                mode='max',
                save_best_only=True,
                verbose=3
            ),
            EarlyStopping(
                monitor='val_f1_m', 
                mode='max',
                patience=PATIENCE_EARLY_STOPPING,
                verbose=2,
                restore_best_weights=True 
            ),
            ReduceLROnPlateau(
                monitor='val_f1_m',
                mode='max',
                factor=0.4,
                patience=PATIENCE_REDUCE_LR,
                min_lr=MIN_LR,
                verbose=2
            ),
        ]

        history_phase1 = model.fit(
            train_gen_phase1,
            epochs=EPOCHS_PHASE1,
            validation_data=val_gen_phase1,
            callbacks=callbacks_phase1,
        )

        print(f"\n--- Фаза 2: Продолжение обучения с Batch Size {BATCH_SIZE_PHASE2} на макс. {EPOCHS_PHASE2_MAX} эпох ---")
        train_gen_phase2 = LandslideDataGenerator(
            train_df_split, folder_path, batch_size=BATCH_SIZE_PHASE2, augment=True, shuffle=True)
        val_gen_phase2 = LandslideDataGenerator(
            val_df_split, folder_path, batch_size=BATCH_SIZE_PHASE2, augment=False, shuffle=False)

        callbacks_phase2 = [
            ModelCheckpoint(
                f"best_model{experiment_name}.h5.keras",
                monitor='val_f1_m',
                mode='max',
                save_best_only=True,
                verbose=3
            ),
            EarlyStopping(
                monitor='val_f1_m',
                mode='max',
                patience=PATIENCE_EARLY_STOPPING,
                restore_best_weights=True,
                verbose=2
            ),
            ReduceLROnPlateau(
                monitor='val_f1_m',
                mode='max',
                factor=0.4,
                patience=PATIENCE_REDUCE_LR,
                min_lr=MIN_LR,
                verbose=2
            ),
        ]

        history_phase2 = model.fit(
            train_gen_phase2,
            epochs=EPOCHS_PHASE2_MAX,
            validation_data=val_gen_phase2,
            callbacks=callbacks_phase2,
        )

        model_save_dir = 'ensemble_models_custom'
        os.makedirs(model_save_dir, exist_ok=True)
        model_save_path = os.path.join(model_save_dir, f'{experiment_name}.keras')
        model.save(model_save_path, save_format='keras')
        print(f"Модель {model_type} (Запуск {i+1}) сохранена в {model_save_path}")

        trained_model_paths.append(model_save_path)
        model_metadata.append({'type': model_type, 'seed': current_seed, 'path': model_save_path})

        experiment.end()

print("\n--- Обучение всех моделей ансамбля завершено ---")

### F1 plot on the validation set for all 6 models from the ensemble (2 each EfficientnetB4, ResNet50, DenseNet169)
![ensamble.png](https://github.com/MakarRybkin/Slide_and_seek_classification/raw/master/images_for_report/ensamble.png)

### Getting predictions for each trained model from the ensemble and averaging the responses

In [ ]:
from tensorflow.keras.models import  load_model
all_predictions = []

print("\n--- Получение предсказаний каждой обученной моделью ---")
for model_path in trained_model_paths:
    print(f"Загрузка модели: {model_path}")

    model = load_model(model_path,compile=False)

    preds = model.predict(val_gen, verbose=1)
    all_predictions.append(preds)

    tf.keras.backend.clear_session()

all_predictions = np.array(all_predictions)

ensemble_probabilities = np.mean(all_predictions, axis=0) 


### Definition of the data generator for the test sample (LandslideTestGenerator). Just like LandslideDataGenerator, it loads data in batches

In [ ]:
class LandslideTestGenerator(Sequence):
    def __init__(self, df, data_path, batch_size=32):
        self.df = df.reset_index(drop=True)
        self.data_path = data_path
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        batch_df = self.df.iloc[index * self.batch_size:(index + 1) * self.batch_size]
        batch_X = []

        for _, row in batch_df.iterrows():
            image_id = row['ID']
            image = load_and_normalize_npy_image(image_id, self.data_path)
            batch_X.append(image)

        return np.array(batch_X)


### Performing predictions on a test sample and creating a file for submission

In [ ]:
test_df = pd.read_csv(test_csv_path)

test_gen = LandslideTestGenerator(test_df, test_data_path, batch_size=32)

all_test_predictions = []

print("Making predictions with each trained model on the Test set:")
for model_path in trained_model_paths:
    print(f"Loading model for test predictions: {model_path}")
    
    model = load_model(model_path, compile=False)

    preds_on_test = model.predict(test_gen, verbose=1)
    all_test_predictions.append(preds_on_test)
    
    tf.keras.backend.clear_session()

all_test_predictions = np.array(all_test_predictions)
ensemble_test_probabilities = np.mean(all_test_predictions, axis=0)

y_test_pred = (ensemble_test_probabilities > 0.5).astype(int)

unique, counts = np.unique(y_test_pred, return_counts=True)
prediction_counts = dict(zip(unique, counts))
print("Prediction counts:", prediction_counts)

submission_df = pd.DataFrame({
    'ID': test_df['ID'],
    'label': y_test_pred.flatten()
})
submission_df.to_csv(f'Submission_File{experiment_name}.csv', index=False)
print(f"Sample submission file created as 'Submission_File{experiment_name}.csv'.")
